In [1]:
import requests
import json
import pandas as pd
import os
import datetime as dt 
from pprint import pprint
%load_ext dotenv
%dotenv secrets.env

Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 8


In [2]:
stations_df = pd.read_csv('data_from_web/weather_stations.csv')
stations_df.head()

,NCDCID,WBAN,NAME,COUNTRY,ST,COUNTY,LAT,LON,ELEV,UTC,STNTYPE,GHCND
0,30014605,64757,EGBERT1W,CANADA,ON,NaN,44.2326,-79.7810,807,-5,USCRN,CAW00064757
1,30121205,23583,ALEKNAGIK1NNE,UNITEDSTATES,AK,DILLINGHAM,59.2840,-158.6145,80,-9,USCRN,NaN
2,30113392,26656,BETHEL87WNW,UNITEDSTATES,AK,NaN,61.3465,-164.0769,153,-9,USCRN,USW00026656
3,30110253,96405,CORDOVA14ESE,UNITEDSTATES,AK,VALDEZ-CORDOVA,60.4731,-145.3542,83,-9,USCRN,USW00096405
4,30094617,26565,DEADHORSE3S,UNITEDSTATES,AK,NORTHSLOPEBOROUGH,70.1618,-148.4644,30,-9,USCRN,USW00026565


In [3]:
stadium_df = pd.read_csv('data_from_web/another_stadiums.csv', delimiter=';')
stadium_df = stadium_df[['TEAM', 'NAME', 'Geo Point', 'ROOF_TYPE']]
stadium_df = stadium_df.append(pd.DataFrame({'TEAM': 'LOSANGELESCHARGERS/LOSANGELESRAMS', 'NAME':'Los Angeles Memorial Coliseum', 'Geo Point': '34.051,-118.1716', 'ROOF_TYPE':'Open'}, index=[31]))
for index1,row1 in stadium_df.iterrows():
    latDif = 10000
    longDif = 10000
    latitude, longitude = row1['Geo Point'].split(',')
    latitude = float(latitude)
    longitude= float(longitude)
    for index2, row2 in stations_df.iterrows():
        if abs(latitude-(row2['LAT'])) < latDif and abs(longitude-row2['LON'])<longDif:
            latDif = abs(latitude-row2['LAT'])
            longDif = abs(longitude-row2['LON'])
            stadium_df.loc[index1 , 'Station'] = row2['GHCND']
team = []
for index, row in stadium_df.iterrows():
    team.append(row['TEAM'].upper().replace(' ',''))
stadium_df['TEAM'] = team
stadium_df.loc[1, 'TEAM'] = 'CHICAGOBEARS'
stadium_df.set_index('TEAM', inplace=True)
stadium_df

,NAME,Geo Point,ROOF_TYPE,Station
TEAM,,,,
DALLASCOWBOYS,Cowboys Stadium,"32.746930527,-97.0923739136",Retractable,USW00053961
CHICAGOBEARS,Soldier Field,"41.8625000675,-87.6167699762",Open,USW00054811
NEWENGLANDPATRIOTS,Gillette Stadium,"42.0918799131,-71.2649100654",Open,USW00054796
CAROLINAPANTHERS,Bank of America Stadium,"35.2258400005,-80.8533099799",Open,USW00092821
TAMPABAYBUCCANEERS,Raymond James Stadium,"27.978840052,-82.5034900566",Open,USW00092827
SEATTLESEAHAWKS,Qwest Stadium,"47.5952976774,-122.331714406",Open,USW00004237
JACKSONVILLEJAGUARS,Jacksonville Municipal Stadium,"30.3238700522,-81.6368999561",Open,USW00063856
HOUSTONTEXANS,Reliant Stadium,"29.6849299406,-95.410919978",Retractable,USW00053960
PHILADELPHIAEAGLES,Lincoln Financial Field,"39.9014700517,-75.1672900567",Open,USW00003761


In [4]:
games_df = pd.read_csv('data_from_web/nfl_2018_games.csv')
games_df['Winner'] = games_df['Winner/tie']
games_df['Loser'] = games_df['Loser/tie']
hometeam = []
awayteam = []
for index,row in games_df.iterrows():
    if row[5]=='@':
        hometeam.append(row['Loser/tie'])
        awayteam.append(row['Winner/tie'])
    else:
        hometeam.append(row['Winner/tie'])
        awayteam.append(row['Loser/tie'])
games_df['Home Team'] = hometeam
games_df['Away Team'] = awayteam
games_df.drop(columns=['Day', 'Unnamed: 5', 'Unnamed: 7', 'Winner/tie', 'Loser/tie', 'TOW', 'TOL'], inplace = True)
games_df.reset_index(inplace=True)
games_df.rename(columns={'index':'Game ID'}, inplace=True)
games_df.set_index('Game ID', inplace=True)
date_obj = []
for index,row in games_df.iterrows():
    date_obj.append(dt.datetime.strptime(row['Date']+' 2018', '%B %d %Y'))
games_df['Datetime Obj'] = date_obj
stadium = []
station = []
for index,row in games_df.iterrows():
    team = row['Home Team'].upper().replace(' ', '')
    if team == 'NEWYORKGIANTS' or team == 'NEWYORKJETS':
        team = 'NEWYORKGIANTS/NEWYORKJETS'
    if team == 'LOSANGELESCHARGERS' or team =='LOSANGELESRAMS':
        team = 'LOSANGELESCHARGERS/LOSANGELESRAMS'
    stadium.append(stadium_df['NAME'].loc[team])
    station.append(stadium_df['Station'].loc[team])
games_df['Stadium'] = stadium
games_df['Station'] = station
games_df


,Week,Date,Time,PtsW,PtsL,YdsW,YdsL,Winner,Loser,Home Team,Away Team,Datetime Obj,Stadium,Station
Game ID,,,,,,,,,,,,,,
0,1,September 6,8:20PM,18,12,232,299,Philadelphia Eagles,Atlanta Falcons,Philadelphia Eagles,Atlanta Falcons,2018-09-06,Lincoln Financial Field,USW00003761
1,1,September 9,1:00PM,48,40,529,475,Tampa Bay Buccaneers,New Orleans Saints,New Orleans Saints,Tampa Bay Buccaneers,2018-09-09,Louisiana Superdome,USW00053960
2,1,September 9,1:00PM,24,16,343,327,Minnesota Vikings,San Francisco 49ers,Minnesota Vikings,San Francisco 49ers,2018-09-09,Hubert H. Humphrey Metrodome,USW00054854
3,1,September 9,1:00PM,27,20,342,336,Miami Dolphins,Tennessee Titans,Miami Dolphins,Tennessee Titans,2018-09-09,Sun Life Stadium,USW00092821
4,1,September 9,1:00PM,20,15,305,324,Jacksonville Jaguars,New York Giants,New York Giants,Jacksonville Jaguars,2018-09-09,Meadowlands Stadium,USW00064756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,17,December 30,4:25PM,26,24,463,426,Baltimore Ravens,Cleveland Browns,Baltimore Ravens,Cleveland Browns,2018-12-30,M&T Bank Stadium,USW00064758
252,17,December 30,4:25PM,35,3,409,292,Kansas City Chiefs,Oakland Raiders,Kansas City Chiefs,Oakland Raiders,2018-12-30,Arrowhead Stadium,USW00023908
253,17,December 30,4:25PM,24,0,360,89,Philadelphia Eagles,Washington Redskins,Washington Redskins,Philadelphia Eagles,2018-12-30,FedEx Field,USW00003761


In [5]:
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2018-09-09&enddate=2018-09-09&stationid=GHCND:CAW00064757'
requests.get(url, headers={"token": os.getenv('NOAA_TOK')}).json()

{'metadata': {'resultset': {'offset': 1, 'count': 3, 'limit': 25}},
 'results': [{'date': '2018-09-09T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:CAW00064757',
   'attributes': ',,R,',
   'value': 0},
  {'date': '2018-09-09T00:00:00',
   'datatype': 'TMAX',
   'station': 'GHCND:CAW00064757',
   'attributes': ',,R,',
   'value': 148},
  {'date': '2018-09-09T00:00:00',
   'datatype': 'TMIN',
   'station': 'GHCND:CAW00064757',
   'attributes': ',,R,',
   'value': 69}]}